# Analyzing Assessed Value of the Downtown Mall

## Goals: 

    *Provide descriptive statistics of assessed value over time for the City of Charlottesville's downtown mall

    *Map and chart assessed values over time

## Step 1: Aquire Data

### Assessment Values
    Charlottesville's Open Data Portal : http://opendata.charlottesville.org/

    Real Estate (All Assessments) Dataset : http://opendata.charlottesville.org/datasets/real-estate-all-assessments

        * On the Real Estate dataset page, in the upper right corner of the window under the map click the APIs             drop down
        * Copy the GeoJSON link
        * Use the GeoJSON link to pull data directly from the Open Data portal using the code below

    Parcel Area Data : http://opendata.charlottesville.org/datasets/parcel-boundary-area

### List of properties to use in analysis

Charlottesville GIS Viewer: https://gisweb.charlottesville.org/GisViewer/

    * Under Map option, turn on 'Parcels & Buildings' > 'Parcels'. Turn everything else off.

    * Zoom to area of interest on map

    * Under 'Tools' select 'Identify'

    * In 'Identify' toolbar select 'Custom Shape' and under 'Layer' select 'Parcels'

    * Using mouse on map, click boundary around area of interest

    * A list will appear in the left panel of the web page

    * In the panel click 'Tools' > 'Export All to Excel'

    * A window named 'Export Results' will open when your download is ready.

    * Click 'View Export' and save file to your project directory

<img src="https://github.com/strmwtr/downtown_assessments/blob/master/img/getting_pin_list.png?raw=true">

# Step 2: Prepare Data

## Looking at the .xls retrieved from the GIS Viewer

In [46]:
#Import pandas module
import pandas as pd 

#Path to the .xls retrieve from the GIS Viewer
f = r'./data/pin_exp.xls'

#Create a dataframe that reads the .xls file
df = pd.read_excel(f)

### Let's look at the first 5 rows of our .xls

In [47]:
df.head()

,FullAddress,OBJECTID,PIN,GPIN,ParcelNumber,OwnerName,CurrentAssessedValue,CurrentTaxYear,CurrentAssessedValueWithLabel,PicturePath,...,MULTIPIN,OwnerAddress,OwnerCityState,OwnerZipCode,SHAPE.STArea(),SHAPE.STLength(),cvGIS.CITY.parcel_area.CreatedBy,cvGIS.CITY.parcel_area.CreatedDate,cvGIS.CITY.parcel_area.ModifiedBy,cvGIS.CITY.parcel_area.ModifiedDate
0,0 3RD ST SE,24846953,280036300,7309,280036300,"LITTLE MOOSE, LLC",204100,2019 Value:,"2019 Value: 204,100",http://realestate.charlottesville.org/IMAGES\P...,...,NaN,P O BOX 4226,CHARLOTTESVILLE VA,22905,1110.750,178.976929,NaN,NaN,NaN,NaN
1,0 E MARKET ST,24848655,330245100,14744,330245100,FIRST AND MAIN CHARLOTTESVILLE LLC,1122200,2019 Value:,"2019 Value: 1,122,200",NaN,...,NaN,224 14TH STREET NW,CHARLOTTESVILLE VA,22903,11316.250,425.844859,NaN,NaN,NaN,NaN
2,0 W MARKET ST & 2ND ST NW,24841086,330262000,6656,330262000,"SPENCER, HAWES, ETAL, TR PROTICO PROP LD TR",418400,2019 Value:,"2019 Value: 418,400",http://realestate.charlottesville.org/IMAGES\P...,...,NaN,700 E HIGH ST,CHARLOTTESVILLE VA,22902,3802.250,254.696684,NaN,NaN,NaN,NaN
3,100 5TH ST SE,24845005,530065300,7426,530065300,"MAIN, RALPH TR OF BLACK DUCK LD TR",664000,2019 Value:,"2019 Value: 664,000",http://realestate.charlottesville.org/IMAGES\P...,...,NaN,P O BOX 2378,CHARLOTTESVILLE VA,22902,787.375,118.075826,NaN,NaN,NaN,NaN
4,100 E MAIN ST,24839773,280020000,7021,280020000,ONE HUNDRED EAST MAIN LTD PART,1904200,2019 Value:,"2019 Value: 1,904,200",http://realestate.charlottesville.org/IMAGES\P...,...,NaN,MSC BOX 5186,CHARLOTTESVILLE VA,22905,5102.750,418.086500,NaN,NaN,NaN,NaN


We can quickly see what is provided by this .xls sheet and the general format of the data it contains. We can see that there are 23 columns by the 5 rows x 23 columns line printed at the end of .head(). 

Looking at the column headers

In [80]:
print("df.columns:\n", df.columns)

print("\ndf['PIN'].head():\n", df['PIN'].head())

print("\ndf['PIN'].shape:\n",df['PIN'].shape)

print("\ndf['PIN'].unique():\n", df['PIN'].unique())

print("\ndf['PIN'].unique().shape:", df['PIN'].unique().shape)



df.columns:
 Index(['FullAddress', 'OBJECTID', 'PIN', 'GPIN', 'ParcelNumber', 'OwnerName',
       'CurrentAssessedValue', 'CurrentTaxYear',
       'CurrentAssessedValueWithLabel', 'PicturePath',
       'CurrentAssessedValueYearLabel', 'CurrentAssessedValueText',
       'MultiParcel', 'MULTIPIN', 'OwnerAddress', 'OwnerCityState',
       'OwnerZipCode', 'SHAPE.STArea()', 'SHAPE.STLength()',
       'cvGIS.CITY.parcel_area.CreatedBy',
       'cvGIS.CITY.parcel_area.CreatedDate',
       'cvGIS.CITY.parcel_area.ModifiedBy',
       'cvGIS.CITY.parcel_area.ModifiedDate'],
      dtype='object')

df['PIN'].head():
 0    280036300
1    330245100
2    330262000
3    530065300
4    280020000
Name: PIN, dtype: object

df['PIN'].shape:
 (207,)

df['PIN'].unique():
 [280036300 330245100 330262000 530065300 280020000 280019000 280020100
 330255000 330248000 530065400 280021000 330244000 '280051A00' 530065500
 280022000 330232000 330256000 530065600 '280051B00' 330241000 280016100
 530058000 530065700 2

Now we have a list of unique PIN's that we can use to build our desired Assessments over time dataframe from. Let's set this to it's own dataframe so we can easily call it in the future

In [7]:
#Create dataframe unique_pins_df from unique PIN's in df  
unique_pins_df = pd.DataFrame(df['PIN'].unique())
#Set column name to PIN
unique_pins_df.columns = ['PIN']
unique_pins_df.head()

,PIN
0,280036300
1,330245100
2,330262000
3,530065300
4,280020000


## Obtaining annual assessment data

Charlottesville's Open Data Portal : http://opendata.charlottesville.org/

Real Estate (All Assessments) Dataset : http://opendata.charlottesville.org/datasets/real-estate-all-assessments

* On the Real Estate dataset page, in the upper right corner of the window under the map click the 'API Explorer' tab
* Copy the Query URL link and augment the link to match your query
* Use the augmented link to pull data directly from the Open Data portal using the code below

In [8]:
# importing the requests library 
import requests

formatted_pins = [f'%27{x}%27' for x in df['PIN'].unique()]
formatted_pins_1 = formatted_pins[:75]
formatted_pins_2 = formatted_pins[75:]
formatted_pins_1 = ','.join(formatted_pins_1)
formatted_pins_2 = ','.join(formatted_pins_2)

url1 = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_1})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"
url2 = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/2/query?where=UPPER(ParcelNumber)%20in%20({formatted_pins_2})%20&outFields=ParcelNumber,LandValue,ImprovementValue,TotalValue,TaxYear&outSR=4326&f=json"

r1 = requests.get(url1)
r2 = requests.get(url2)

d1 = r1.json()
d2 = r2.json()
print(r1,r2)

<Response [200]> <Response [200]>


After testing the requests.get(url), I can request up to 120 parcels at a time before recieving a 404 Error. I have 126 parcels of interest. I will break my request up into 2 parts, [:75] and [75:].

On a future date I will write a function that checks for responses under 400 based on length of results. This will eliminate the need for writing out formatted_pins_1 .. 2 .. 3. 

Check data type

In [82]:
#Check data type
print(type(d1))

print()
#Check keys
print(d1.keys())

print()
#check features key
print(d1['features'][:5])

print()
#Check type of d1['features']
type(d1['features'])



<class 'dict'>

dict_keys(['displayFieldName', 'fieldAliases', 'fields', 'features'])

[{'attributes': {'ParcelNumber': '280010000', 'LandValue': 2191100, 'ImprovementValue': 3812400, 'TotalValue': 6003500, 'TaxYear': '2019'}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 2140600, 'ImprovementValue': 4038400, 'TotalValue': 6179000, 'TaxYear': '2018'}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 2140600, 'ImprovementValue': 4051208, 'TotalValue': 6191808, 'TaxYear': '2017'}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 1719500, 'ImprovementValue': 4271200, 'TotalValue': 5990700, 'TaxYear': '2016'}}, {'attributes': {'ParcelNumber': '280010000', 'LandValue': 1563200, 'ImprovementValue': 1120000, 'TotalValue': 2683200, 'TaxYear': '2015'}}]



list

## Create date series based on features and combine data frames into a single df

In [61]:
df1 = pd.DataFrame(d1['features'])
df2 = pd.DataFrame(d2['features'])
print('.shape of df1, df2: ', df1.shape, df2.shape)
df1 = df1.append(df2, ignore_index = True)
print('.shape of df1 after appending df2: ', df1.shape)

print('\ndf1.head(): \n',df1.head())

print('\ndf1.keys(): ',df1.keys())

print('\ntype(df1["attributes"]): ',type(df1['attributes']))

.shape of df1, df2:  (1683, 1) (1173, 1)
.shape of df1 after appending df2:  (2856, 1)

df1.head(): 
                                           attributes
0  {'ParcelNumber': '280010000', 'LandValue': 219...
1  {'ParcelNumber': '280010000', 'LandValue': 214...
2  {'ParcelNumber': '280010000', 'LandValue': 214...
3  {'ParcelNumber': '280010000', 'LandValue': 171...
4  {'ParcelNumber': '280010000', 'LandValue': 156...

df1.keys():  Index(['attributes'], dtype='object')

type(df1["attributes"]):  <class 'pandas.core.series.Series'>


## Create a single data frame based on combined series data

In [20]:
assessments = pd.DataFrame([x for x in df1['attributes']])
assessments.head()

,ImprovementValue,LandValue,ParcelNumber,TaxYear,TotalValue
0,3812400,2191100,280010000,2019,6003500
1,4038400,2140600,280010000,2018,6179000
2,4051208,2140600,280010000,2017,6191808
3,4271200,1719500,280010000,2016,5990700
4,1120000,1563200,280010000,2015,2683200


In [72]:
print('assessments.keys():\n',assessments.keys())

print('\nassessments.shape: ', assessments.shape)

print('\nassessments.isnull().any():\n', assessments.isnull().any())

assessments.keys():
 Index(['ImprovementValue', 'LandValue', 'ParcelNumber', 'TaxYear',
       'TotalValue'],
      dtype='object')

assessments.shape:  (2856, 5)

assessments.isnull().any():
 ImprovementValue    False
LandValue           False
ParcelNumber        False
TaxYear             False
TotalValue          False
dtype: bool


print("\nassessments['ParcelNumber'].describe()\n", assessments['ParcelNumber'].describe())

print("\nassessments['TaxYear'].describe()\n", assessments['TaxYear'].describe())

print("\nassessments['TaxYear'].min(), assessments['TaxYear'].max()\n", assessments['TaxYear'].min(), assessments['TaxYear'].max())

print("\nassessments['ImprovementValue'].describe()\n", assessments['ImprovementValue'].describe())

print("\nassessments['LandValue'].describe()\n", assessments['LandValue'].describe())

print("\nassessments['TotalValue'].describe()\n", assessments['TotalValue'].describe())

taxyearmin = assessments['TaxYear'] == assessments['TaxYear'].min()
assessments[taxyearmin].describe()

taxyearmax = assessments['TaxYear'] == assessments['TaxYear'].max()
assessments[taxyearmax].describe()

In [32]:
assessments[taxyearmax].describe()-assessments[taxyearmin].describe()

,ImprovementValue,LandValue,TotalValue
count,4.000000e+00,4.000000e+00,4.000000e+00
mean,1.393978e+06,6.268242e+05,2.020802e+06
std,3.013812e+06,8.424608e+05,3.736914e+06
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.859750e+05,2.045750e+05,5.463250e+05
50%,5.081500e+05,3.607500e+05,8.688500e+05
75%,1.172425e+06,6.276750e+05,1.717575e+06
max,2.722880e+07,4.611500e+06,3.184030e+07


## Access json file for parcel areas

import folium

formatted_gpins = [str(x) for x in df['GPIN'].unique()]
formatted_gpins = formatted_gpins
formatted_gpins = ','.join(formatted_gpins)

parcel_area_url = f"https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_1/MapServer/43/query?where=GPIN%20in%20({formatted_gpins})&outFields=*&outSR=4326&f=json"

print(parcel_area_url)

m = folium.Map(location=[38.0309,-78.4804],tiles='Stamen Terrain',zoom_start=17)
style_function = {'fillOpacity': 0, 'fillColor': '#black'}
folium.GeoJson(parcel_area_url,name='Parcels', style_function=style_function).add_to(m)

folium.LayerControl().add_to(m)
m



m = folium.Map(location=[38.0309,-78.4804],tiles='Stamen Terrain',zoom_start=17)

folium.Choropleth(
    geo_data=parcel_area_url,
    name='choropleth',
    data=assessments,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [35]:
df_key = df[['PIN','GPIN']]
df_key

,PIN,GPIN
0,280036300,7309
1,330245100,14744
2,330262000,6656
3,530065300,7426
4,280020000,7021
5,280019000,6984
6,280020100,7197
7,330255000,6819
8,330248000,6876
9,530065400,7447


In [36]:
assessments.head()

,ImprovementValue,LandValue,ParcelNumber,TaxYear,TotalValue
0,3812400,2191100,280010000,2019,6003500
1,4038400,2140600,280010000,2018,6179000
2,4051208,2140600,280010000,2017,6191808
3,4271200,1719500,280010000,2016,5990700
4,1120000,1563200,280010000,2015,2683200


In [40]:
pin_count = df_key.groupby('PIN').count() 
pin_count

,GPIN
PIN,
280001000,1
280010000,1
280012000,1
280013000,1
280016000,10
280016100,1
280018000,1
280019000,1
280020000,1


In [45]:
multi_pin = pin_count[pin_count.GPIN != 1]
multi_pin

,GPIN
PIN,
280016000,10
330257100,4
530054101,27
530055000,2
530064000,2
530072000,23
530091000,17
280037A00,4
